In [2]:
!pip install sgp4
!pip install spacetrack
!pip install pandas
!pip install astropy

In [3]:
import datetime
from sgp4.api import jday
from sgp4.api import Satrec
import pandas as pd

import spacetrack.operators as op
from spacetrack import SpaceTrackClient

st = SpaceTrackClient(identity='hitalo.c.a@gmail.com', password='EuMeChamoHitalo2000!')

In [4]:
!ls

sample_data  tle_latest.txt


In [5]:
data = st.tle_latest(iter_lines=True, ordinal=1, epoch='>now-30',
                     mean_motion=op.inclusive_range(2, 5),
                     eccentricity=op.less_than(0.01), format='tle')

with open('tle_latest.txt', 'w') as fp:
    for line in data:
        fp.write(line + '\n')

In [6]:
tle_latest = pd.read_csv("tle_latest.txt", header=None)

s_array = tle_latest[tle_latest.index%2 == 0].values.tolist()
t_array = tle_latest[tle_latest.index%2 != 0].values.tolist()

In [7]:
s_array[0][0]
t_array[0][0]

'2 07369 125.1211  60.0806 0068766 332.6582 344.8546  3.07215005529779'

In [8]:
s = '1 25544U 98067A   19343.69339541  .00001764  00000-0  38792-4 0  9991'
t = '2 25544  51.6439 211.2001 0007417  17.6667  85.6398 15.50103472202482'
s = s_array[0][0]
t = t_array[0][0]
satellite = Satrec.twoline2rv(s, t)

current_time = datetime.datetime.now()
jd, fr = jday(current_time.year, current_time.month, current_time.day, current_time.hour, current_time.minute, current_time.second)

e, r, v = satellite.sgp4(jd, fr)
print(e)

print(r)  # True Equator Mean Equinox position (km)

print(v)  # True Equator Mean Equinox velocity (km/s)
from sgp4.api import SGP4_ERRORS
print(SGP4_ERRORS)

0
(11923.129644787927, -1786.9044453955078, 15977.489604591025)
(-1.6568741147021395, -4.058091744141413, 0.8193698952354813)
{1: 'mean eccentricity is outside the range 0.0 to 1.0', 2: 'nm is less than zero', 3: 'perturbed eccentricity is outside the range 0.0 to 1.0', 4: 'semilatus rectum is less than zero', 5: '(error 5 no longer in use; it meant the satellite was underground)', 6: 'mrt is less than 1.0 which indicates the satellite has decayed'}


## Convert position in True Equator Mean Equinox (TEME) to Latitude and Longitude { World Geodetic System (WGS)}

In [26]:
from astropy.time import Time
################ IMPORTANTE ##################################
# Esse jd é o tempo obtido na célula anterior
##############################################################
t = Time(jd, format='jd')
print(t)
error_code, teme_p, teme_v = satellite.sgp4(t.jd1, t.jd2)  # in km and km/s
if error_code != 0:
    raise RuntimeError(SGP4_ERRORS[error_code])
teme_p = r
teme_v = v

from astropy.coordinates import TEME, CartesianDifferential, CartesianRepresentation
from astropy import units as u
teme_p = CartesianRepresentation(teme_p*u.km)
teme_v = CartesianDifferential(teme_v*u.km/u.s)
teme = TEME(teme_p.with_differentials(teme_v), obstime=t)

from astropy.coordinates import ITRS
itrs = teme.transform_to(ITRS(obstime=t))  
location = itrs.earth_location
location.geodetic  

2459489.5


GeodeticLocation(lon=<Longitude -19.45918017 deg>, lat=<Latitude 53.02131736 deg>, height=<Quantity 13651.34061498 km>)

In [10]:
 #jd

2459489.5

In [23]:
print(location.geodetic.lon)

-19d27m33.04861868s


In [15]:
!wget https://gist.githubusercontent.com/MatheusCod/995376a7e0b40e31534ffe08894f8bdd/raw/455234a61c1b4dd95340d28b334b2479c9d07e6c/helloWorldWind.js
!wget https://gist.githubusercontent.com/MatheusCod/995376a7e0b40e31534ffe08894f8bdd/raw/455234a61c1b4dd95340d28b334b2479c9d07e6c/index.html

--2021-10-02 17:39:28--  https://gist.githubusercontent.com/MatheusCod/995376a7e0b40e31534ffe08894f8bdd/raw/455234a61c1b4dd95340d28b334b2479c9d07e6c/helloWorldWind.js
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1239 (1.2K) [text/plain]
Saving to: ‘helloWorldWind.js’

helloWorldWind.js   100%[===================>]   1.21K  --.-KB/s    in 0s      

2021-10-02 17:39:29 (49.7 MB/s) - ‘helloWorldWind.js’ saved [1239/1239]

--2021-10-02 17:39:29--  https://gist.githubusercontent.com/MatheusCod/995376a7e0b40e31534ffe08894f8bdd/raw/455234a61c1b4dd95340d28b334b2479c9d07e6c/index.html
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubu

In [19]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(80)"))

https://8rex9i93s3x-496ff2e9c6d22116-80-colab.googleusercontent.com/


In [17]:
!python -m http.server 8000

Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [02/Oct/2021 17:39:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2021 17:39:46] "GET /helloWorldWind.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Oct/2021 17:39:47] code 404, message File not found
127.0.0.1 - - [02/Oct/2021 17:39:47] "GET /favicon.ico HTTP/1.1" 404 -

Keyboard interrupt received, exiting.
^C
